# Set up the environment

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

# import tf
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import tensorflow.keras.backend as K

# import os functions
import os
import time

import numpy as np
import matplotlib.pyplot as plt

from IPython import display

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

1 Physical GPUs, 1 Logical GPUs


# Load the data

### Read the files

In [4]:
X_test = np.load("./EEG_data/X_test.npy")
y_test = np.load("./EEG_data/y_test.npy") - 769
person_train_valid = np.load("./EEG_data/person_train_valid.npy")
X_train_valid = np.load("./EEG_data/X_train_valid.npy")
y_train_valid = np.load("./EEG_data/y_train_valid.npy") - 769
person_test = np.load("./EEG_data/person_test.npy")

### Shape of data

In [5]:
print("training/Valid data shape: {}".format(X_train_valid.shape))       # training data of many persons
print("Test data shape: {}".format(X_test.shape))                        # test data of many persons
print("Training/Valid target shape: {}".format(y_train_valid.shape))     # training labels of many persons
print("Test target shape: {}".format(y_test.shape))                      # test labels of many persons
print("Person train/valid  shape: {}".format(person_train_valid.shape))  # which person correspond to the trail in test set
print("Person test shape: {}".format(person_test.shape))                 # which person correspond to the trail in test set

training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid  shape: (2115, 1)
Person test shape: (443, 1)


### divide dataset into training and validation

In [6]:
perm = np.random.permutation(X_train_valid.shape[0])
num_train = int(0.8 * X_train_valid.shape[0])
num_valid = X_train_valid.shape[0] - num_train
X_train =  X_train_valid[perm[0:num_train]]
y_train =  y_train_valid[perm[0:num_train]]
X_valid = X_train_valid[perm[num_train: ]]
y_valid = y_train_valid[perm[num_train: ]]


print("Training data shape: {}".format(X_train.shape))
print("Training label shape: {}".format(y_train.shape))
print("Validation data shape: {}".format(X_valid.shape))
print("Validation label shape: {}".format(y_valid.shape))
print("Test data shape: {}".format(X_test.shape))
print("Test label shape: {}".format(y_test.shape))

Training data shape: (1692, 22, 1000)
Training label shape: (1692,)
Validation data shape: (423, 22, 1000)
Validation label shape: (423,)
Test data shape: (443, 22, 1000)
Test label shape: (443,)


### Preprocess data

In [7]:
def sliding_window(X_arr, y_arr, time_window=100, time_step=1, time_stride=1):
    temp_x = np.moveaxis(X_arr, 2, 0)
    temp_x = temp_x.astype(np.float32)
    buff = []
    
    num_slices = (len(temp_x)-time_window*time_step) // time_stride + 1
    
    # get time slices for data
    for i in range(num_slices):
        buff.append(temp_x[i*time_stride:i*time_stride + time_window*time_step:time_step])
        buff[i] = np.moveaxis(buff[i], 0, 2)
        # uncomment this if additional dimension is needed
        # buff[i] = buff[i].reshape(1, buff[i].shape[0], buff[i].shape[1], buff[i].shape[2])
        
    temp_x = np.concatenate(buff)
        
    # get time slice for labels
    temp_y = np.ones((X_arr.shape[0],num_slices))
    
    for i in range(len(y_arr)):
        temp_y[i] = temp_y[i] * y_arr[i]
        
    temp_y = temp_y.reshape((-1))
    
    return temp_x, temp_y

# Experiment 1: CRNN

In this experiment we make a model inspired by "Recurrent Deep Neural Networks for Real-Time Sleep Stage Classification From Single Channel EEG" by E. Bresch, 2018. We reduce kernel size of the filters because our data has less timestamps.

### Construct model

In [55]:
def construct_CRNN_model(TIME_WINDOW, dropout=0, regularizer=0):
    # input
    crnn_input = layers.Input(shape=(22, TIME_WINDOW))


    # ================================== CONV1 ================================== #

    t1 = tf.keras.layers.Permute((2, 1))(crnn_input)
    c1 = layers.Conv1D(32, 4, strides=1)(t1)
    new_size = TIME_WINDOW - 8 + 1
    
    bn1 = layers.BatchNormalization(axis=1)(c1)
    a1 = layers.Activation("relu")(bn1)
    do1 = layers.Dropout(0.5)(a1)
    maxpool1 = layers.MaxPooling1D(4)(do1)
    new_size = new_size//8 + 1
    
    # =========================================================================== #

    # ================================== CONV2 ================================== #

    c2 = layers.Conv1D(64, 4, strides=1)(maxpool1)
    new_size = new_size - 8 + 1
    
    bn2 = layers.BatchNormalization(axis=1)(c2)
    a2 = layers.Activation("relu")(bn2)
    do2 = layers.Dropout(0.5)(a2)
    maxpool2 = layers.MaxPooling1D(4)(do2)
    new_size = new_size//8 + 1
    
    # =========================================================================== #


    # ================================== LSTM ================================== #

    lstm3 = layers.LSTM(64, 
                        return_sequences=True, 
                        dropout=dropout, 
                        kernel_regularizer=keras.regularizers.l2(regularizer))(maxpool2)
    lstm4 = layers.LSTM(64, 
                        return_sequences=True, 
                        dropout=dropout,
                        kernel_regularizer=keras.regularizers.l2(regularizer),
                        recurrent_constraint=keras.regularizers.l2(regularizer),
                        activity_regularizer=keras.regularizers.l2(regularizer))(lstm3)

    
    # =========================================================================== #

    f7 = layers.Flatten()(lstm4)

    # output
    crnn_output = layers.Dense(4, activation="softmax")(f7)
    
    return keras.Model(inputs = crnn_input, outputs = crnn_output)

In [34]:
crnn_model_1000 = construct_CRNN_model(1000, dropout=0.3, regularizer=0.001)
crnn_model_1000.compile("adam", "sparse_categorical_crossentropy", metrics=["acc"])

In [35]:
crnn_model_1000.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 22, 1000)]        0         
_________________________________________________________________
permute_9 (Permute)          (None, 1000, 22)          0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 997, 32)           2848      
_________________________________________________________________
batch_normalization_8 (Batch (None, 997, 32)           3988      
_________________________________________________________________
activation_8 (Activation)    (None, 997, 32)           0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 997, 32)           0         
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 249, 32)           0   

### Make checkpoints

In [36]:
# save model with the best accuracy 
checkpoint_callback = [
    keras.callbacks.ModelCheckpoint(
        filepath='./model_checkpoints/crnn_1000',
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        save_best_only=True,
        monitor='val_loss',
        verbose=1)
]### Make checkpoints

### Train model

In [37]:
crnn_model_1000_loss_hist = crnn_model_1000.fit(X_train, y_train,
                                                validation_data = (X_valid, y_valid),
                                                epochs = 10,
                                                callbacks=checkpoint_callback)

Train on 1692 samples, validate on 423 samples
Epoch 1/10
1664/1692 [============================>.] - ETA: 0s - loss: 1.5707 - acc: 0.3047
Epoch 00001: val_loss improved from inf to 1.50053, saving model to ./model_checkpoints/crnn_1000
INFO:tensorflow:Assets written to: ./model_checkpoints/crnn_1000\assets
1692/1692 [==============================] - 14s 8ms/sample - loss: 1.5689 - acc: 0.3073 - val_loss: 1.5005 - val_acc: 0.3617
Epoch 2/10
1664/1692 [============================>.] - ETA: 0s - loss: 1.4721 - acc: 0.3552
Epoch 00002: val_loss improved from 1.50053 to 1.44891, saving model to ./model_checkpoints/crnn_1000
INFO:tensorflow:Assets written to: ./model_checkpoints/crnn_1000\assets
1692/1692 [==============================] - 12s 7ms/sample - loss: 1.4718 - acc: 0.3546 - val_loss: 1.4489 - val_acc: 0.3806
Epoch 3/10
1664/1692 [============================>.] - ETA: 0s - loss: 1.4006 - acc: 0.4159
Epoch 00003: val_loss improved from 1.44891 to 1.35931, saving model to ./mode

### Log

crnn_model_1000 = construct_CRNN_model(1000, dropout=0.3, regularizer=0.001) 55 % at best

added activity regularizer.

aug_crnn_model_1000 = construct_CRNN_model(1000, dropout=0.3, regularizer=0.01) 43% at best

aug_crnn_model_1000 = construct_CRNN_model(1000, dropout=0.5, regularizer=0.005) 53% at best

changed relu to elu, addded a new dense elu layer in the end, addded new lstm layer

construct_aug_CRNN_model(1000, dropout=0.3, regularizer=0.005)

# Experiment 2: augmented CRNN

In this experiment we add a dense layer in the end. Intuitively, it should help to form better features out of LSTM output. We also add one more layer for improved resolution.

### Construct model

In [111]:
def construct_aug_CRNN_model(TIME_WINDOW, dropout=0, regularizer=0):
    # input
    crnn_input = layers.Input(shape=(22, TIME_WINDOW))


    # ================================== CONV1 ================================== #

    t1 = tf.keras.layers.Permute((2, 1))(crnn_input)
    c1 = layers.Conv1D(32, 4, strides=1)(t1)
    new_size = TIME_WINDOW - 8 + 1
    
    bn1 = layers.BatchNormalization(axis=1)(c1)
    a1 = layers.Activation("elu")(bn1)
    do1 = layers.Dropout(0.5)(a1)
    maxpool1 = layers.MaxPooling1D(4)(do1)
    new_size = new_size//8 + 1
    
    # =========================================================================== #

    # ================================== CONV2 ================================== #

    c2 = layers.Conv1D(64, 4, strides=1)(maxpool1)
    new_size = new_size - 8 + 1
    
    bn2 = layers.BatchNormalization(axis=1)(c2)
    a2 = layers.Activation("elu")(bn2)
    do2 = layers.Dropout(0.5)(a2)
    maxpool2 = layers.MaxPooling1D(4)(do2)
    new_size = new_size//8 + 1
    
    # =========================================================================== #


    # ================================== LSTM ================================== #

    lstm3 = layers.LSTM(64, 
                        return_sequences=True, 
                        dropout=dropout, 
                        kernel_regularizer=keras.regularizers.l2(regularizer),
                        activity_regularizer=keras.regularizers.l2(regularizer))(maxpool2)
    lstm4 = layers.LSTM(64, 
                        return_sequences=True, 
                        dropout=dropout,
                        kernel_regularizer=keras.regularizers.l2(regularizer),
                        activity_regularizer=keras.regularizers.l2(regularizer))(lstm3)
    
    lstm5 = layers.LSTM(16, 
                        return_sequences=True, 
                        dropout=dropout,
                        kernel_regularizer=keras.regularizers.l2(regularizer),
                        activity_regularizer=keras.regularizers.l2(regularizer))(lstm4)


    
    # =========================================================================== #

    f7 = layers.Flatten()(lstm5)
    
    elu7 = layers.Dense(64 * new_size, activation="elu", kernel_regularizer=keras.regularizers.l2(regularizer))(f7)

    # output
    crnn_output = layers.Dense(4, activation="softmax")(elu7)
    
    return keras.Model(inputs = crnn_input, outputs = crnn_output)

In [115]:
aug_crnn_model_1000.compile(keras.optimizers.Adam(), "sparse_categorical_crossentropy", metrics=["acc"])
aug_crnn_model_1000 = construct_aug_CRNN_model(1000, dropout=0.3, regularizer=0.001)

In [116]:
aug_crnn_model_1000.summary()

Model: "model_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_29 (InputLayer)        [(None, 22, 1000)]        0         
_________________________________________________________________
permute_29 (Permute)         (None, 1000, 22)          0         
_________________________________________________________________
conv1d_50 (Conv1D)           (None, 997, 32)           2848      
_________________________________________________________________
batch_normalization_48 (Batc (None, 997, 32)           3988      
_________________________________________________________________
activation_48 (Activation)   (None, 997, 32)           0         
_________________________________________________________________
dropout_48 (Dropout)         (None, 997, 32)           0         
_________________________________________________________________
max_pooling1d_47 (MaxPooling (None, 249, 32)           0  

### Make checkpoints

In [ ]:
# save model with the best accuracy 
checkpoint_callback = [
    keras.callbacks.ModelCheckpoint(
        filepath='./model_checkpoints/aug_crnn_1000',
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        save_best_only=True,
        monitor='val_loss',
        verbose=1)
]### Make checkpoints

### Train model

In [121]:
aug_crnn_model_1000.fit(X_train, y_train,
                        validation_data = (X_valid, y_valid),
                        epochs = 10,
                        callbacks = checkpoint_callback)

Train on 1692 samples, validate on 423 samples
Epoch 1/10
1664/1692 [============================>.] - ETA: 0s - loss: 0.9223 - acc: 0.6707
Epoch 00001: val_loss did not improve from 1.10012
1692/1692 [==============================] - 26s 15ms/sample - loss: 0.9258 - acc: 0.6684 - val_loss: 1.1466 - val_acc: 0.5721
Epoch 2/10
1664/1692 [============================>.] - ETA: 0s - loss: 0.9012 - acc: 0.6863
Epoch 00002: val_loss did not improve from 1.10012
1692/1692 [==============================] - 18s 11ms/sample - loss: 0.8974 - acc: 0.6891 - val_loss: 1.1482 - val_acc: 0.5556
Epoch 3/10
1664/1692 [============================>.] - ETA: 0s - loss: 0.8814 - acc: 0.6965
Epoch 00003: val_loss did not improve from 1.10012
1692/1692 [==============================] - 18s 11ms/sample - loss: 0.8788 - acc: 0.6974 - val_loss: 1.1237 - val_acc: 0.5792
Epoch 4/10
1664/1692 [============================>.] - ETA: 0s - loss: 0.8812 - acc: 0.6845
Epoch 00004: val_loss did not improve from 1.10

# Experiment 3: optimization of augmented CRNN

In this experiment we vary parameters of CRNN to achieve better accuracy

### Construct model

In [131]:
def construct_opt_aug_CRNN_model(TIME_WINDOW,
                                 filter_size=4,
                                 dropout=0.3, 
                                 regularizer=0.001, 
                                 last_lstm_size=16, 
                                 last_hidden_layer=1000,
                                 last_dropout=0.3):
    # input
    crnn_input = layers.Input(shape=(22, TIME_WINDOW))


    # ================================== CONV1 ================================== #

    t1 = tf.keras.layers.Permute((2, 1))(crnn_input)
    c1 = layers.Conv1D(32, filter_size, strides=1)(t1)
    new_size = TIME_WINDOW - filter_size + 1
    
    bn1 = layers.BatchNormalization(axis=1)(c1)
    a1 = layers.Activation("elu")(bn1)
    do1 = layers.Dropout(0.4)(a1)
    maxpool1 = layers.MaxPooling1D(filter_size)(do1)
    new_size = new_size//filter_size + 1
    
    # =========================================================================== #

    # ================================== CONV2 ================================== #

    c2 = layers.Conv1D(64, filter_size, strides=1)(maxpool1)
    new_size = new_size - filter_size + 1
    
    bn2 = layers.BatchNormalization(axis=1)(c2)
    a2 = layers.Activation("elu")(bn2)
    do2 = layers.Dropout(0.4)(a2)
    maxpool2 = layers.MaxPooling1D(filter_size)(do2)
    new_size = new_size//filter_size + 1
    
    # =========================================================================== #


    # ================================== LSTM ================================== #

    lstm3 = layers.LSTM(64, 
                        return_sequences=True, 
                        dropout=dropout, 
                        kernel_regularizer=keras.regularizers.l2(regularizer),
                        activity_regularizer=keras.regularizers.l2(regularizer))(maxpool2)
    lstm4 = layers.LSTM(64, 
                        return_sequences=True, 
                        dropout=dropout,
                        kernel_regularizer=keras.regularizers.l2(regularizer),
                        activity_regularizer=keras.regularizers.l2(regularizer))(lstm3)
    
    lstm5 = layers.LSTM(last_lstm_size, 
                        return_sequences=True, 
                        dropout=dropout,
                        kernel_regularizer=keras.regularizers.l2(regularizer),
                        activity_regularizer=keras.regularizers.l2(regularizer))(lstm4)


    
    # =========================================================================== #

    f7 = layers.Flatten()(lstm5)
    
    elu7 = layers.Dense(last_hidden_layer, activation="elu", kernel_regularizer=keras.regularizers.l2(regularizer))(f7)
    do7 = layers.Dropout(last_dropout)(elu7)

    # output
    crnn_output = layers.Dense(4, activation="softmax")(do7)
    
    return keras.Model(inputs = crnn_input, outputs = crnn_output)

SyntaxError: invalid syntax (<ipython-input-131-9c95cd4297fe>, line 71)

### Vary parameters

In [130]:
TIME_WINDOW = 1000


# save model with the best accuracy 
checkpoint_callback = [
    keras.callbacks.ModelCheckpoint(
        filepath='./model_checkpoints/opt_aug_crnn_1000_32' ,
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        save_best_only=True,
        monitor='val_loss',
        verbose=1)
]

opt_aug_crnn_1000 = construct_opt_aug_CRNN_model(TIME_WINDOW, 
                                                 dropout=0.3, 
                                                 regularizer=0.001, 
                                                 last_lstm_size=32, 
                                                 last_hidden_layer=1000,
                                                 last_dropout=0.3)

opt_aug_crnn_1000.compile("adam", "sparse_categorical_crossentropy", metrics=["acc"])

opt_aug_crnn_1000.fit(X_train, y_train,
                      validation_data = (X_valid, y_valid),
                      epochs = 75,
                      callbacks=checkpoint_callback)

Train on 1692 samples, validate on 423 samples
Epoch 1/75
1664/1692 [============================>.] - ETA: 0s - loss: 2.9761 - acc: 0.2680
Epoch 00001: val_loss improved from inf to 2.52723, saving model to ./model_checkpoints/opt_aug_crnn_1000_32
INFO:tensorflow:Assets written to: ./model_checkpoints/opt_aug_crnn_1000_32\assets
1692/1692 [==============================] - 33s 20ms/sample - loss: 2.9697 - acc: 0.2701 - val_loss: 2.5272 - val_acc: 0.3310
Epoch 2/75
1664/1692 [============================>.] - ETA: 0s - loss: 2.2934 - acc: 0.3522
Epoch 00002: val_loss improved from 2.52723 to 2.07675, saving model to ./model_checkpoints/opt_aug_crnn_1000_32
INFO:tensorflow:Assets written to: ./model_checkpoints/opt_aug_crnn_1000_32\assets
1692/1692 [==============================] - 32s 19ms/sample - loss: 2.2898 - acc: 0.3522 - val_loss: 2.0767 - val_acc: 0.3499
Epoch 3/75
1664/1692 [============================>.] - ETA: 0s - loss: 1.9435 - acc: 0.3504
Epoch 00003: val_loss improved f

KeyboardInterrupt: 

In [ ]:
TIME_WINDOW = 1000


# save model with the best accuracy 
checkpoint_callback = [
    keras.callbacks.ModelCheckpoint(
        filepath='./model_checkpoints/opt_aug_crnn_1000_64' ,
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        save_best_only=True,
        monitor='val_loss',
        verbose=1)
]

opt_aug_crnn_1000 = construct_opt_aug_CRNN_model(TIME_WINDOW, 
                                                 dropout=0.3, 
                                                 regularizer=0.001, 
                                                 last_lstm_size=64, 
                                                 last_hidden_layer=1000,
                                                 last_dropout=0.3)

opt_aug_crnn_1000.compile("adam", "sparse_categorical_crossentropy", metrics=["acc"])

opt_aug_crnn_1000.fit(X_train, y_train,
                      validation_data = (X_valid, y_valid),
                      epochs = 75,
                      callbacks=checkpoint_callback)

# NEW EXPERIMENTS HERE
Vary last_lstm_size,  last_hidden_layer, last_dropout and TIME WINDOW. 
Leave dropout=0.3 and regularization=0.001

In [8]:
from tensorboard.plugins.hparams import api as hp

In [18]:
HP_TIME_WINDOW = hp.HParam('TIME_WINDOW', hp.Discrete([750, 1000])) 
HP_BATCH_SIZE = hp.HParam('BATCH_SIZE', hp.Discrete([32])) 
HP_HIDDEN = hp.HParam('HIDDEN', hp.Discrete([128]))
HP_DROPOUT = hp.HParam('DROPOUT', hp.Discrete([0.3]))
HP_REGULARIZER = hp.HParam('REGULARIZER', hp.Discrete([.001]))
HP_last_lstm_size = hp.HParam('LAST_LSTM', hp.Discrete([64, 128]))
HP_last_hidden_layer = hp.HParam('LAST_HIDDEN', hp.Discrete([500, 1000]))
HP_last_dropout = hp.HParam('LAST_DROPOUT', hp.Discrete([0.2, 0.3]))
HP_LEARNING = hp.HParam('LEARNING', hp.Discrete([.0001]))
HP_BETA = hp.HParam('BETA', hp.Discrete([.9]))
VAL_ACCURACY = 'val_accuracy'
TEST_ACCURACY = 'test accuracy'


X_train_norm = X_train - np.mean(X_train)/np.std(X_train)
X_valid_norm = X_valid - np.mean(X_train)/np.std(X_train)
X_test_norm = X_test - np.mean(X_train)/np.std(X_train)

with tf.summary.create_file_writer('logs/crnn_hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_TIME_WINDOW, 
#              HP_BATCH_SIZE, 
#              HP_HIDDEN, 
#              HP_DROPOUT, 
#              HP_REGULARIZER, 
             HP_last_lstm_size,
             HP_last_hidden_layer,
             HP_last_dropout,
#              HP_LEARNING, 
#              HP_BETA
            ],
    metrics=[hp.Metric(VAL_ACCURACY, display_name='Val Accuracy'), 
             hp.Metric(TEST_ACCURACY, display_name='Test Accuracy')],
)

In [19]:
def construct_opt_aug_CRNN_model(hparams,
                                 filter_size=4):
    # input
    crnn_input = layers.Input(shape=(22, hparams[HP_TIME_WINDOW]))


    # ================================== CONV1 ================================== #

    t1 = tf.keras.layers.Permute((2, 1))(crnn_input)
    c1 = layers.Conv1D(32, filter_size, strides=1)(t1)
    new_size = hparams[HP_TIME_WINDOW] - filter_size + 1
    
    bn1 = layers.BatchNormalization(axis=1)(c1)
    a1 = layers.Activation("elu")(bn1)
    do1 = layers.Dropout(hparams[HP_DROPOUT])(a1)
    maxpool1 = layers.MaxPooling1D(filter_size)(do1)
    new_size = new_size//filter_size + 1
    
    # =========================================================================== #

    # ================================== CONV2 ================================== #

    c2 = layers.Conv1D(64, filter_size, strides=1)(maxpool1)
    new_size = new_size - filter_size + 1
    
    bn2 = layers.BatchNormalization(axis=1)(c2)
    a2 = layers.Activation("elu")(bn2)
    do2 = layers.Dropout(hparams[HP_DROPOUT])(a2)
    maxpool2 = layers.MaxPooling1D(filter_size)(do2)
    new_size = new_size//filter_size + 1
    
    # =========================================================================== #


    # ================================== LSTM ================================== #

    lstm3 = layers.LSTM(hparams[HP_HIDDEN], 
                        return_sequences=True, 
                        dropout=hparams[HP_DROPOUT], 
                        kernel_regularizer=keras.regularizers.l2(hparams[HP_REGULARIZER]),
                        activity_regularizer=keras.regularizers.l2(hparams[HP_REGULARIZER]))(maxpool2)
    lstm4 = layers.LSTM(hparams[HP_HIDDEN], 
                        return_sequences=True, 
                        dropout=hparams[HP_DROPOUT],
                        kernel_regularizer=keras.regularizers.l2(hparams[HP_REGULARIZER]),
                        activity_regularizer=keras.regularizers.l2(hparams[HP_REGULARIZER]))(lstm3)
    
    lstm5 = layers.LSTM(hparams[HP_last_lstm_size], 
                        return_sequences=True, 
                        dropout=hparams[HP_DROPOUT],
                        kernel_regularizer=keras.regularizers.l2(hparams[HP_REGULARIZER]),
                        activity_regularizer=keras.regularizers.l2(hparams[HP_REGULARIZER]))(lstm4)


    
    # =========================================================================== #

    f7 = layers.Flatten()(lstm5)
    
    elu7 = layers.Dense(hparams[HP_last_hidden_layer], activation="elu", kernel_regularizer=keras.regularizers.l2(hparams[HP_REGULARIZER]))(f7)
    do7 = layers.Dropout(hparams[HP_last_dropout])(elu7)

    # output
    crnn_output = layers.Dense(4, activation="softmax")(do7)
    
    return keras.Model(inputs = crnn_input, outputs = crnn_output)

In [20]:
# save model with the best accuracy 
checkpoint_callback = [
    keras.callbacks.ModelCheckpoint(
        filepath='./model_checkpoints/crnn_lstm_model' ,
        # Path where to save the model
        # The two parameters below mean that we will overwrite
        # the current checkpoint if and only if
        # the `val_loss` score has improved.
        save_best_only=True,
        monitor='val_loss',
        verbose=1)
]

def train_test_crnn_model(hparams):
    rnn_lstm_model = construct_opt_aug_CRNN_model(hparams)
    
    adam = keras.optimizers.Adam(learning_rate=hparams[HP_LEARNING], beta_1=hparams[HP_BETA])

    rnn_lstm_model.compile(optimizer=adam, loss="sparse_categorical_crossentropy", metrics=["acc"])
    rnn_lstm_model.summary()
    hist = rnn_lstm_model.fit(X_train_slices, y_train_slices, validation_data = (X_valid_slices, y_valid_slices), 
                              epochs = 100, callbacks=checkpoint_callback, batch_size=hparams[HP_BATCH_SIZE]).history
    _, accuracy = rnn_lstm_model.evaluate(X_test_slices, y_test_slices)
    return accuracy, hist, rnn_lstm_model


def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    test_accuracy, hist, model = train_test_crnn_model(hparams)
    tf.summary.scalar(VAL_ACCURACY, max(hist['val_acc']), step=1)
    tf.summary.scalar(TEST_ACCURACY, test_accuracy, step=1)
  return model

In [21]:
session_num = 0
TIME_STRIDE = 1000
for window_size in HP_TIME_WINDOW.domain.values:
            
    X_train_slices, y_train_slices = sliding_window(X_train_norm, 
                                                    y_train, 
                                                    time_window=window_size,  
                                                    time_stride=TIME_STRIDE)


    X_valid_slices, y_valid_slices = sliding_window(X_valid_norm, 
                                                    y_valid, 
                                                    time_window=window_size, 
                                                    time_stride=TIME_STRIDE)

    X_test_slices, y_test_slices = sliding_window(X_test_norm, 
                                                    y_test, 
                                                    time_window=window_size, 
                                                    time_stride=TIME_STRIDE)

    print("Training data shape with slices: {}".format(X_train_slices.shape))
    print("Training label shape with slice: {}".format(y_train_slices.shape))
    print("Validation data shape with slices: {}".format(X_valid_slices.shape))
    print("Validation label shape with slice: {}".format(y_valid_slices.shape))
    print("Testing data shape with slices: {}".format(X_test_slices.shape))
    print("Testing label shape with slice: {}".format(y_test_slices.shape))
    for num_units in HP_HIDDEN.domain.values:
        for batch_size in HP_BATCH_SIZE.domain.values:
            for dropout_rate in HP_DROPOUT.domain.values:
                for last_lstm in HP_last_lstm_size.domain.values:
                    for last_num_units in HP_last_hidden_layer.domain.values:
                        for last_dropout_rate in HP_last_dropout.domain.values:
                            for reg in HP_REGULARIZER.domain.values:
                                for learning_rate in HP_LEARNING.domain.values:
                                    for beta in HP_BETA.domain.values:
                                        hparams = {
                                            HP_TIME_WINDOW: window_size,
                                            HP_BATCH_SIZE: batch_size,
                                            HP_HIDDEN: num_units,
                                            HP_DROPOUT: dropout_rate,
                                            HP_REGULARIZER: reg,
                                            HP_last_lstm_size: last_lstm,
                                            HP_last_hidden_layer: last_num_units,
                                            HP_last_dropout: last_dropout_rate,
                                            HP_LEARNING: learning_rate,
                                            HP_BETA: beta,
                                        }

                                        run_name = "run-%d" % session_num
                                        print('--- Starting trial: %s' % run_name)
                                        print({h.name: hparams[h] for h in hparams})
                                        model = run('logs/crnn_hparam_tuning/' + run_name, hparams)
                                        session_num += 1


Training data shape with slices: (1692, 22, 750)
Training label shape with slice: (1692,)
Validation data shape with slices: (423, 22, 750)
Validation label shape with slice: (423,)
Testing data shape with slices: (443, 22, 750)
Testing label shape with slice: (443,)
--- Starting trial: run-0
{'TIME_WINDOW': 750, 'BATCH_SIZE': 32, 'HIDDEN': 128, 'DROPOUT': 0.3, 'REGULARIZER': 0.001, 'LAST_LSTM': 64, 'LAST_HIDDEN': 500, 'LAST_DROPOUT': 0.2, 'LEARNING': 0.0001, 'BETA': 0.9}
Model: "model_85"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_86 (InputLayer)        [(None, 22, 750)]         0         
_________________________________________________________________
permute_85 (Permute)         (None, 750, 22)           0         
_________________________________________________________________
conv1d_170 (Conv1D)          (None, 747, 32)           2848      
___________________________________________

Epoch 13/100
1568/1692 [==========================>...] - ETA: 0s - loss: 1.8130 - acc: 0.5121
Epoch 00013: val_loss improved from 1.81159 to 1.80665, saving model to ./model_checkpoints/crnn_lstm_model
INFO:tensorflow:Assets written to: ./model_checkpoints/crnn_lstm_model/assets
1692/1692 [==============================] - 8s 5ms/sample - loss: 1.8159 - acc: 0.5065 - val_loss: 1.8067 - val_acc: 0.4752
Epoch 14/100
1600/1692 [===========================>..] - ETA: 0s - loss: 1.7615 - acc: 0.5356
Epoch 00014: val_loss improved from 1.80665 to 1.72905, saving model to ./model_checkpoints/crnn_lstm_model
INFO:tensorflow:Assets written to: ./model_checkpoints/crnn_lstm_model/assets
1692/1692 [==============================] - 9s 5ms/sample - loss: 1.7663 - acc: 0.5313 - val_loss: 1.7290 - val_acc: 0.5201
Epoch 15/100
1600/1692 [===========================>..] - ETA: 0s - loss: 1.7208 - acc: 0.5181
Epoch 00015: val_loss improved from 1.72905 to 1.72069, saving model to ./model_checkpoints/c

1664/1692 [============================>.] - ETA: 0s - loss: 1.2855 - acc: 0.6052
Epoch 00034: val_loss improved from 1.36088 to 1.34766, saving model to ./model_checkpoints/crnn_lstm_model
INFO:tensorflow:Assets written to: ./model_checkpoints/crnn_lstm_model/assets
1692/1692 [==============================] - 8s 5ms/sample - loss: 1.2846 - acc: 0.6058 - val_loss: 1.3477 - val_acc: 0.5887
Epoch 35/100
1568/1692 [==========================>...] - ETA: 0s - loss: 1.2792 - acc: 0.6167
Epoch 00035: val_loss did not improve from 1.34766
1692/1692 [==============================] - 1s 513us/sample - loss: 1.2816 - acc: 0.6135 - val_loss: 1.3585 - val_acc: 0.5650
Epoch 36/100
1632/1692 [===========================>..] - ETA: 0s - loss: 1.2451 - acc: 0.6225
Epoch 00036: val_loss improved from 1.34766 to 1.29982, saving model to ./model_checkpoints/crnn_lstm_model
INFO:tensorflow:Assets written to: ./model_checkpoints/crnn_lstm_model/assets
1692/1692 [==============================] - 9s 6ms/s

Epoch 59/100
1664/1692 [============================>.] - ETA: 0s - loss: 1.0384 - acc: 0.6995
Epoch 00059: val_loss did not improve from 1.19543
1692/1692 [==============================] - 1s 397us/sample - loss: 1.0391 - acc: 0.7004 - val_loss: 1.2278 - val_acc: 0.6430
Epoch 60/100
1568/1692 [==========================>...] - ETA: 0s - loss: 1.0171 - acc: 0.7085
Epoch 00060: val_loss did not improve from 1.19543
1692/1692 [==============================] - 1s 425us/sample - loss: 1.0221 - acc: 0.7033 - val_loss: 1.2570 - val_acc: 0.6288
Epoch 61/100
1568/1692 [==========================>...] - ETA: 0s - loss: 1.0292 - acc: 0.6869
Epoch 00061: val_loss did not improve from 1.19543
1692/1692 [==============================] - 1s 429us/sample - loss: 1.0212 - acc: 0.6915 - val_loss: 1.2312 - val_acc: 0.6265
Epoch 62/100
1664/1692 [============================>.] - ETA: 0s - loss: 0.9851 - acc: 0.6929
Epoch 00062: val_loss did not improve from 1.19543
1692/1692 [========================

1600/1692 [===========================>..] - ETA: 0s - loss: 0.8488 - acc: 0.7525
Epoch 00088: val_loss did not improve from 1.17312
1692/1692 [==============================] - 1s 549us/sample - loss: 0.8494 - acc: 0.7512 - val_loss: 1.2550 - val_acc: 0.6430
Epoch 89/100
1568/1692 [==========================>...] - ETA: 0s - loss: 0.8538 - acc: 0.7570
Epoch 00089: val_loss did not improve from 1.17312
1692/1692 [==============================] - 1s 559us/sample - loss: 0.8505 - acc: 0.7589 - val_loss: 1.3095 - val_acc: 0.6430
Epoch 90/100
1568/1692 [==========================>...] - ETA: 0s - loss: 0.8756 - acc: 0.7423
Epoch 00090: val_loss did not improve from 1.17312
1692/1692 [==============================] - 1s 569us/sample - loss: 0.8753 - acc: 0.7441 - val_loss: 1.2046 - val_acc: 0.6643
Epoch 91/100
1632/1692 [===========================>..] - ETA: 0s - loss: 0.8550 - acc: 0.7506
Epoch 00091: val_loss did not improve from 1.17312
1692/1692 [==============================] - 1s 

...

1600/1692 [===========================>..] - ETA: 0s - loss: 0.9345 - acc: 0.7188
Epoch 00077: val_loss did not improve from 1.15013
1692/1692 [==============================] - 1s 577us/sample - loss: 0.9260 - acc: 0.7240 - val_loss: 1.1881 - val_acc: 0.6690
Epoch 78/100
1632/1692 [===========================>..] - ETA: 0s - loss: 0.8573 - acc: 0.7567
Epoch 00078: val_loss did not improve from 1.15013
1692/1692 [==============================] - 1s 582us/sample - loss: 0.8590 - acc: 0.7577 - val_loss: 1.2360 - val_acc: 0.6903
Epoch 79/100
1568/1692 [==========================>...] - ETA: 0s - loss: 0.8762 - acc: 0.7430
Epoch 00079: val_loss did not improve from 1.15013
1692/1692 [==============================] - 1s 573us/sample - loss: 0.8845 - acc: 0.7411 - val_loss: 1.2370 - val_acc: 0.6856
Epoch 80/100
1600/1692 [===========================>..] - ETA: 0s - loss: 0.8789 - acc: 0.7506
Epoch 00080: val_loss did not improve from 1.15013
1692/1692 [==============================] - 1s 

1632/1692 [===========================>..] - ETA: 0s - loss: 4.0662 - acc: 0.2880
Epoch 00001: val_loss did not improve from 1.15013
1692/1692 [==============================] - 5s 3ms/sample - loss: 4.0528 - acc: 0.2926 - val_loss: 3.7023 - val_acc: 0.3546
Epoch 2/100
1664/1692 [============================>.] - ETA: 0s - loss: 3.5784 - acc: 0.3558
Epoch 00002: val_loss did not improve from 1.15013
1692/1692 [==============================] - 1s 585us/sample - loss: 3.5763 - acc: 0.3582 - val_loss: 3.3836 - val_acc: 0.3475
Epoch 3/100
1632/1692 [===========================>..] - ETA: 0s - loss: 3.3387 - acc: 0.3824
Epoch 00003: val_loss did not improve from 1.15013
1692/1692 [==============================] - 1s 588us/sample - loss: 3.3352 - acc: 0.3830 - val_loss: 3.1711 - val_acc: 0.3570
Epoch 4/100
1568/1692 [==========================>...] - ETA: 0s - loss: 3.1345 - acc: 0.4011
Epoch 00004: val_loss did not improve from 1.15013
1692/1692 [==============================] - 1s 580us

1632/1692 [===========================>..] - ETA: 0s - loss: 1.2912 - acc: 0.6005
Epoch 00031: val_loss did not improve from 1.15013
1692/1692 [==============================] - 1s 567us/sample - loss: 1.2964 - acc: 0.5969 - val_loss: 1.4351 - val_acc: 0.5414
Epoch 32/100
1664/1692 [============================>.] - ETA: 0s - loss: 1.2896 - acc: 0.6010
Epoch 00032: val_loss did not improve from 1.15013
1692/1692 [==============================] - 1s 551us/sample - loss: 1.2855 - acc: 0.6034 - val_loss: 1.4108 - val_acc: 0.5485
Epoch 33/100
1600/1692 [===========================>..] - ETA: 0s - loss: 1.2485 - acc: 0.5975
Epoch 00033: val_loss did not improve from 1.15013
1692/1692 [==============================] - 1s 579us/sample - loss: 1.2411 - acc: 0.6046 - val_loss: 1.4731 - val_acc: 0.5603
Epoch 34/100
1600/1692 [===========================>..] - ETA: 0s - loss: 1.2627 - acc: 0.6156
Epoch 00034: val_loss did not improve from 1.15013
1692/1692 [==============================] - 1s 

1664/1692 [============================>.] - ETA: 0s - loss: 1.0039 - acc: 0.7079
Epoch 00061: val_loss did not improve from 1.15013
1692/1692 [==============================] - 1s 585us/sample - loss: 1.0041 - acc: 0.7074 - val_loss: 1.3448 - val_acc: 0.6028
Epoch 62/100
1664/1692 [============================>.] - ETA: 0s - loss: 1.0113 - acc: 0.6977
Epoch 00062: val_loss did not improve from 1.15013
1692/1692 [==============================] - 1s 595us/sample - loss: 1.0062 - acc: 0.7004 - val_loss: 1.4149 - val_acc: 0.5816
Epoch 63/100
1664/1692 [============================>.] - ETA: 0s - loss: 0.9766 - acc: 0.7145
Epoch 00063: val_loss did not improve from 1.15013
1692/1692 [==============================] - 1s 590us/sample - loss: 0.9771 - acc: 0.7134 - val_loss: 1.4804 - val_acc: 0.5745
Epoch 64/100
1632/1692 [===========================>..] - ETA: 0s - loss: 0.9958 - acc: 0.7126
Epoch 00064: val_loss did not improve from 1.15013
1692/1692 [==============================] - 1s 

1600/1692 [===========================>..] - ETA: 0s - loss: 0.8312 - acc: 0.7887
Epoch 00091: val_loss did not improve from 1.15013
1692/1692 [==============================] - 1s 607us/sample - loss: 0.8293 - acc: 0.7884 - val_loss: 1.5887 - val_acc: 0.6241
Epoch 92/100
1600/1692 [===========================>..] - ETA: 0s - loss: 0.8381 - acc: 0.7675
Epoch 00092: val_loss did not improve from 1.15013
1692/1692 [==============================] - 1s 592us/sample - loss: 0.8349 - acc: 0.7701 - val_loss: 1.7309 - val_acc: 0.5934
Epoch 93/100
1664/1692 [============================>.] - ETA: 0s - loss: 0.8749 - acc: 0.7590
Epoch 00093: val_loss did not improve from 1.15013
1692/1692 [==============================] - 1s 601us/sample - loss: 0.8764 - acc: 0.7595 - val_loss: 1.4885 - val_acc: 0.6336
Epoch 94/100
1568/1692 [==========================>...] - ETA: 0s - loss: 0.8346 - acc: 0.7704
Epoch 00094: val_loss did not improve from 1.15013
1692/1692 [==============================] - 1s 

In [17]:
%reload_ext tensorboard
%tensorboard --logdir logs/crnn_hparam_tuning

Reusing TensorBoard on port 6008 (pid 9483), started 0:08:28 ago. (Use '!kill 9483' to kill it.)

In [14]:
!kill 27448
